In [1]:
import numpy as np 
import pandas as pd

### ESTIMATE

#### estimate score

In [3]:
sig = pd.read_excel("/Users/qiluzhou/Desktop/umass/research/score/data/stromal_immune_sig.xlsx", header = 0, sheet_name = 0)
# immune_sig_est = list(sig.loc[2,:])[2:]
# stromal_sig_est = list(sig.loc[1,:])[2:]

/Users/qiluzhou/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [17]:
data = pd.read_table("/Users/qiluzhou/Desktop/umass/research/score/data/chol_tcga.txt", sep = "\t")
# remove NA and duplicate names
data = data.drop_duplicates(subset = ['Hugo_Symbol'])
data = data.dropna(subset = "Hugo_Symbol", axis = 0)
#data.to_csv("/Users/qiluzhou/Desktop/umass/research/score/data/chol_tcga_clean.txt", sep = "\t")

In [35]:
# rank normalization
row_names = data.Hugo_Symbol # gene names
df = data.iloc[:, 2:]

df_ranked = df.apply(pd.to_numeric).rank(axis = 0, method = "average")
df_ranked.index = row_names
df_normalized = 10000*df_ranked/df_ranked.shape[0]
gene_set_names = list(sig.iloc[1:3, 0])
score = np.empty((df_ranked.shape[1], 2))

In [36]:
# gsea
for i in range(0,2):
    
    gene_set = sig.iloc[i+1,2:]
    common = list(set(gene_set) & set(row_names))

    if len(common) == 0 : 
        continue

    ES_vector = [0]*df_normalized.shape[1]

    for sample in range(0, df_normalized.shape[1]):
        ordered_genes = df_normalized.iloc[:,sample].sort_values(ascending = False)

        hit_ind = ordered_genes.index.isin(gene_set)
        no_hit_ind = 1- hit_ind

        ordered_genes1 = ordered_genes.pow(1./4)
        hit_exp = ordered_genes1[hit_ind]

        no_hit_penalty = np.cumsum(no_hit_ind/sum(no_hit_ind))

        hit_reward = np.cumsum((hit_ind*ordered_genes1)/sum(hit_exp))

        ES_vector[sample] = sum(hit_reward - no_hit_penalty)
    
    score[:, i] = ES_vector

In [41]:
# get estimate score
score = pd.DataFrame(score)
score.index = data.columns[2:]
score.columns = ["stromal", "immune"]
score['estimate'] = score['stromal'] + score['immune']

In [51]:
# get tumor purity(Affymetrix-only)
purity = np.cos(0.6049872018 + 0.0001467884*score['estimate'])

In [49]:
score.head()

,stromal,immune,estimate
TCGA-3X-AAV9-01,3480.865820,4655.945959,8136.811779
TCGA-3X-AAVA-01,1426.604986,2067.395840,3494.000827
TCGA-3X-AAVB-01,3261.592173,3056.319093,6317.911267
TCGA-3X-AAVC-01,937.550265,2459.131260,3396.681525
TCGA-3X-AAVE-01,1680.669364,4245.019743,5925.689107


### ISTME